In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from nltk.tokenize import word_tokenize


In [2]:
train = pd.read_csv('train.tsv', sep="\t")
test = pd.read_csv('test.tsv', sep="\t")




In [3]:
sub = pd.read_csv('sampleSubmission.csv')


In [4]:
test["Sentiment"] = -10
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,-10
1,156062,8545,An intermittently pleasing but mostly routine ...,-10
2,156063,8545,An,-10
3,156064,8545,intermittently pleasing but mostly routine effort,-10
4,156065,8545,intermittently pleasing but mostly routine,-10


In [5]:
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
not_delete = ["but", "shan't", "wasn't", "couldn't", "didn't", "hadn't", "against", "no", "haven't", "shouldn't", "needn't", "wouldn't", "aren't", "mightn't", "won't", "isn't", "hasn't", "don't", "mustn't", "doesn't", "not"]
stop_words = [w for w in stop_words if w not in not_delete]
print(stop_words)

df=pd.concat([train, test], ignore_index = True)

df = df.applymap(lambda s:s.lower() if type(s) == str else s)
#del test, train

['these', 'those', 'then', 't', 'over', 'that', 'be', 'off', 'shouldn', 'been', 'shan', 'll', 'should', 'and', 'needn', 'with', 'in', 'under', 'our', 'own', 'when', "you've", 'very', 'itself', 'which', 'haven', 'd', 'am', 'here', 'such', 'are', 'other', 'ourselves', 'on', 'we', 'wouldn', 'hers', 'herself', 'than', 'up', 'as', 'had', 'do', 'same', 'mustn', 'or', 'while', 're', 'being', 'my', "weren't", 'why', 'if', 'o', "you'd", 'out', 'nor', 'were', 'down', 'hadn', 'doing', 'will', 'mightn', 'into', 'where', 'y', 'because', 'm', 'at', 'its', 'by', 'between', 'further', 'them', "that'll", 'an', 'through', 'theirs', 'weren', 'your', 've', 'who', "she's", 'their', 'any', 'having', 'of', 'yours', 'yourself', 'a', 'about', 's', 'just', 'me', 'once', 'couldn', 'ours', 'yourselves', 'it', 'each', 'was', 'he', 'below', 'what', 'only', 'too', 'don', "you're", 'her', 'does', 'from', 'there', 'himself', 'during', 'myself', 'wasn', 'again', 'they', 'won', 'all', 'more', 'some', 'so', "it's", 'can'

In [6]:
porter = PorterStemmer()
def stem_sentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        if word not in stop_words:
            stem_sentence.append(porter.stem(word))
            stem_sentence.append(" ")
    return "".join(stem_sentence)

In [7]:
def clean_review(phrase):
    clean_review = [ ]
    for i in range(0, len(phrase)):
        review = str(phrase[i])
        review = re.sub("[^a-zA-Z]", " ", review)
        stem_review = stem_sentence(review)
        clean_review.append(stem_review)
    return clean_review

In [8]:
df['clean_review'] = clean_review(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,a series of escapades demonstrating the adage ...,1,seri escapad demonstr adag good goos also good...
1,2,1,a series of escapades demonstrating the adage ...,2,seri escapad demonstr adag good goos
2,3,1,a series,2,seri
3,4,1,a,2,
4,5,1,series,2,seri


In [9]:
df.shape

(222352, 5)

In [10]:
drop_list = []
for i in range(0, len(df.Phrase)):
    if len(df.clean_review[i].split()) < 2:
        drop_list.append(i)
df = df.drop(drop_list, axis = 0)

df.shape

(171314, 5)

In [11]:
test = df[df.Sentiment == -10]
test.drop("Sentiment", axis =1, inplace = True)
print(test.shape)
test.head()

(48686, 4)


/Users/Junda/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,an intermittently pleasing but mostly routine ...,intermitt pleas but mostli routin effort
156061,156062,8545,an intermittently pleasing but mostly routine ...,intermitt pleas but mostli routin effort
156063,156064,8545,intermittently pleasing but mostly routine effort,intermitt pleas but mostli routin effort
156064,156065,8545,intermittently pleasing but mostly routine,intermitt pleas but mostli routin
156065,156066,8545,intermittently pleasing but,intermitt pleas but


In [12]:
train = df[df.Sentiment != -10]
print(train.shape)



(122628, 5)


In [13]:
whole_text = list(df.clean_review.values)
sen_len = []
for text in whole_text:
    word = word_tokenize(text)
    sen_len.append(len(word))
max_len = np.max(sen_len)
max_len

30

In [14]:
from keras.utils import to_categorical
X_test = test.clean_review.values


Using TensorFlow backend.


In [15]:
from sklearn.model_selection import train_test_split
training_set, testing_set = train_test_split(train, test_size = 0.2)
train_set, val_set = train_test_split(train, test_size = 0.2)
X_train = train_set.clean_review.values
Y_train = to_categorical(train_set.Sentiment.values)
X_val = val_set.clean_review.values
Y_val = to_categorical(val_set.Sentiment.values)
X_test = testing_set.clean_review.values
Y_test = to_categorical(testing_set.Sentiment.values)

In [16]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(98102,)
(98102, 5)
(24526,)
(24526, 5)


In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [18]:
tk = Tokenizer(lower=True, filters='')
tk.fit_on_texts(whole_text)
X_train = tk.texts_to_sequences(X_train)
X_val = tk.texts_to_sequences(X_val)
X_test = tk.texts_to_sequences(X_test)

In [19]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_val = pad_sequences(X_val, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print(X_train.shape, X_val.shape, X_test.shape)

(98102, 30) (24526, 30) (24526, 30)


In [20]:
y_test = testing_set.Sentiment.values

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras import backend
from keras.optimizers import Adam

In [ ]:

model1 = Sequential()
model1.add(Embedding(15000, 128))
model1.add(LSTM(64, dropout = 0.4, recurrent_dropout=0.4, return_sequences=True))
model1.add(LSTM(32, dropout = 0.5, recurrent_dropout = 0.5, return_sequences=False))
model1.add(Dense(5, activation='softmax'))
opt = Adam(lr=0.001, decay=1e-5)
model1.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model1.summary()

In [ ]:
model1.fit(X_train, Y_train, batch_size = 32, epochs = 3, verbose = 1, validation_data=(X_val, Y_val))

In [ ]:
p1 = model1.predict_classes(X_test, verbose = 1)


In [22]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dropout

In [29]:

model2= Sequential()
model2.add(Embedding(15000,128,input_length=max_len))
model2.add(Dropout(0.3))

model2.add(Conv1D(64,kernel_size=2,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.3))

model2.add(Dense(5,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 128)           1920000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 30, 64)            24640     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
Total para

In [30]:
model2.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=3, batch_size=32, verbose=1)

Train on 98102 samples, validate on 24526 samples
Epoch 1/3
 9088/98102 [=>............................] - ETA: 45s - loss: 1.3524 - acc: 0.4507

KeyboardInterrupt: 

In [ ]:
p2 = model2.predict_classes(X_test, verbose = 1)

In [ ]:
count2 = 0
for i in range(0, len(p2)):
    if(p2[i]!=y_test[i]):
        count2 += 1

error2 = count2/len(p2)
print(error2)